In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
df = spark.read.option("header", "true").option("inferSchema", "true")

df = spark.table("workspace.default.telco_assignment")
display(df)
 

 


In [0]:
df.show(5)

In [0]:
df_select = df.select('customerID', 'MonthlyCharges')
df_select.show()

In [0]:
contract_count = df.groupBy('Contract').count().toPandas()
import matplotlib.pyplot as plt

plt.figure()
plt.pie(contract_count['count'],
        labels = contract_count['Contract'],
        autopct = '%1.1f%%'
        )
plt.title('Contract Type Proportions')
plt.show()



In [0]:
from pyspark.sql.functions import col
multiple_lines_count = (
    df.filter(col('MultipleLines') == 'Yes').count()
)
multiple_lines_count



In [0]:
from pyspark.sql.functions import min, max

tenure_stats = df.select(
    min("tenure").alias("min_tenure"),
    max("tenure").alias("max_tenure")
).first()

min_tenure = tenure_stats["min_tenure"]
max_tenure = tenure_stats["max_tenure"]

min_tenure, max_tenure


In [0]:
from pyspark.sql.functions import avg
monthly_charges_ongender = (
    df.groupBy('gender').agg(avg('MonthlyCharges').alias('avg_monthly_charges'))
)
monthly_charges_ongender.show()

In [0]:
yearly_avg = (
    df.filter(col('Contract')=='One year').agg(avg('MonthlyCharges')).first()[0]
)

other_avg = (
    df.filter(col('Contract') != 'One year').agg(avg('MonthlyCharges')).first()[0]
)

yearly_pays_more = yearly_avg > other_avg

yearly_pays_more

In [0]:
from pyspark.sql.functions import when

df= df.withColumn(
    'AvarageCharges', when(col('tenure') > 0, col('TotalCharges') / col('tenure')).otherwise(none)
)
df.select('customerID','AverageCharges', 'TotalCharges', 'tenure').show(5)
